<a href="https://colab.research.google.com/github/sungkwangsong/EasyOCR/blob/master/notebooks/KorSciBERT_%ED%8C%A8%EC%B9%98%EC%99%80_%EC%82%AC%EC%9A%A9_%EC%98%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
HBN_MODEL_DIR_PATH='/content/drive/MyDrive/Workspaces/Hibrainnet/models/KoSciBERT'
HBN_DATASET_DIR_PATH='/content/drive/MyDrive/Workspaces/Hibrainnet/datasets/samples'

In [ ]:
cd /content/drive/MyDrive/models/KoSciBERT

/content/drive/MyDrive/models/KoSciBERT


In [ ]:
# KoNLPy 및 의존성 설치
!pip install konlpy

# Java 설치
!apt-get update -qq
!apt-get install -y openjdk-8-jdk python-dev python3-dev

# Mecab 설치
!pip install mecab-python3
!apt-get install -y curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 설치 확인
import konlpy
from konlpy.tag import Mecab
print("KoNLPy 버전:", konlpy.__version__)
print("Mecab 설치 완료")

In [ ]:
# 필요한 패키지 임포트
import os
import sys
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from tqdm import tqdm

class KorSciBertSearch:
    def __init__(self, model_dir, max_seq_length=128):
        """
        KorSci BERT 토크나이저를 활용한 논문 검색 시스템 초기화

        Args:
            model_dir: KorSci BERT 모델이 저장된 디렉토리 경로
            max_seq_length: 최대 시퀀스 길이
        """
        self.model_dir = model_dir
        self.max_seq_length = max_seq_length
        self.papers = None
        self.paper_embeddings = None

        # 필요한 파일 경로 설정
        self.vocab_file = os.path.join(model_dir, "vocab_kisti.txt")

        # 모델 디렉토리를 시스템 경로에 추가 (파일 임포트를 위해)
        if model_dir not in sys.path:
            sys.path.append(model_dir)

        # 토크나이저 초기화
        self._init_tokenizer()

    def _init_tokenizer(self):
        """KorSci 토크나이저 초기화"""
        try:
            # 토크나이저 모듈 직접 임포트
            sys.path.append(self.model_dir)
            import tokenization_kisti as tokenization

            # 토크나이저 초기화 (반드시 Mecab 사용)
            self.tokenizer = tokenization.FullTokenizer(
                vocab_file=self.vocab_file,
                do_lower_case=False,
                tokenizer_type="Mecab"
            )
            print("KISTI 토크나이저 초기화 완료 (Mecab)")
        except Exception as e:
            # 첫 번째 오류 메시지 기록
            first_error = f"Mecab 토크나이저 초기화 오류: {e}"
            print(first_error)

            # 기존 토크나이저 모듈을 재로드
            if 'tokenization_kisti' in sys.modules:
                del sys.modules['tokenization_kisti']

            try:
                # tokenization_kisti.py 파일을 명시적으로 찾아 임포트
                tokenization_path = os.path.join(self.model_dir, 'tokenization_kisti.py')
                if os.path.exists(tokenization_path):
                    import importlib.util
                    spec = importlib.util.spec_from_file_location("tokenization_kisti", tokenization_path)
                    tokenization = importlib.util.module_from_spec(spec)
                    spec.loader.exec_module(tokenization)

                    # WordPiece 토크나이저로 대체
                    self.tokenizer = tokenization.FullTokenizer(
                        vocab_file=self.vocab_file,
                        do_lower_case=False,
                        tokenizer_type="WordPiece"
                    )
                    print("KISTI 토크나이저 초기화 완료 (WordPiece)")
                else:
                    raise FileNotFoundError(f"토크나이저 파일을 찾을 수 없습니다: {tokenization_path}")
            except Exception as e2:
                # 두 번째 오류 메시지 기록 및 예외 발생
                print(f"WordPiece 토크나이저 초기화 오류: {e2}")
                print(f"첫 번째 시도 오류: {first_error}")
                raise RuntimeError("KISTI 토크나이저 초기화에 실패했습니다.")

    def load_papers(self, papers_path):
        """
        논문 데이터 로드

        Args:
            papers_path: 논문 데이터 파일 경로 (JSON 또는 CSV)
        """
        file_ext = os.path.splitext(papers_path)[1].lower()

        if file_ext == '.json':
            with open(papers_path, 'r', encoding='utf-8') as f:
                self.papers = json.load(f)
        elif file_ext == '.csv':
            self.papers = pd.read_csv(papers_path).to_dict('records')
        else:
            raise ValueError(f"지원되지 않는 파일 형식입니다: {file_ext}")

        print(f"총 {len(self.papers)}개의 논문 데이터를 로드했습니다.")

    def _prepare_tfidf_corpus(self):
        """토크나이저를 활용한 TF-IDF 코퍼스 준비"""
        # 모든 논문 텍스트 토큰화
        corpus = []
        for paper in self.papers:
            title = paper.get('title', '')
            abstract = paper.get('abstract', '')
            keywords = ' '.join(paper.get('keywords', []))
            paper_text = title + " " + abstract + " " + keywords
            paper_tokens = self.tokenizer.tokenize(paper_text)
            corpus.append(" ".join(paper_tokens))

        # TF-IDF 벡터화
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(corpus)

        return corpus

    def create_paper_embeddings(self):
        """토크나이저와 TF-IDF를 활용한 논문 임베딩 생성"""
        if self.papers is None:
            raise ValueError("먼저 load_papers()를 통해 논문 데이터를 로드해야 합니다.")

        print("논문 임베딩 생성 중...")

        # TF-IDF 코퍼스 준비 및 벡터화
        self._prepare_tfidf_corpus()

        # TF-IDF 행렬을 numpy 배열로 변환
        self.paper_embeddings = self.tfidf_matrix.toarray()

        print(f"임베딩 생성 완료. 형태: {self.paper_embeddings.shape}")

    def get_embedding(self, text):
        """
        주어진 텍스트의 TF-IDF 임베딩을 추출

        Args:
            text: 임베딩을 추출할 텍스트

        Returns:
            np.ndarray: 임베딩 벡터
        """
        # 토큰화
        tokens = self.tokenizer.tokenize(text)
        tokenized_text = " ".join(tokens)

        # TF-IDF 벡터화
        vector = self.tfidf_vectorizer.transform([tokenized_text]).toarray()[0]

        return vector

    def search(self, query, top_k=10):
        """
        쿼리와 가장 유사한 논문을 검색

        Args:
            query: 검색 쿼리 텍스트
            top_k: 반환할 상위 결과 수

        Returns:
            list: 관련 논문 목록 (유사도 내림차순)
        """
        if self.papers is None or self.paper_embeddings is None:
            raise ValueError("검색하기 전에 먼저 논문 데이터를 로드하고 임베딩을 생성해야 합니다.")

        # 쿼리 임베딩 계산
        query_embedding = self.get_embedding(query)

        # 코사인 유사도 계산
        similarities = cosine_similarity([query_embedding], self.paper_embeddings)[0]

        # 유사도에 따라 정렬된 인덱스 얻기
        top_indices = np.argsort(similarities)[::-1][:top_k]

        # 결과 구성
        results = []
        for idx in top_indices:
            results.append({
                'paper': self.papers[idx],
                'similarity': similarities[idx]
            })

        return results

    def close(self):
        """리소스 해제 (필요시)"""
        pass

In [ ]:
import json

def create_sample_papers():
    """샘플 논문 데이터 생성"""
    return [
        {
            "id": "paper1",
            "title": "한국어 자연어처리를 위한 BERT 기반 임베딩 모델 연구",
            "abstract": "본 연구에서는 한국어 자연어처리를 위한 BERT 기반 임베딩 모델을 제안한다. 제안된 모델은 대용량 한국어 코퍼스를 사용하여 사전학습되었으며, 다양한 자연어처리 태스크에서 우수한 성능을 보인다.",
            "authors": ["김철수", "이영희"],
            "year": 2022,
            "keywords": ["자연어처리", "BERT", "임베딩", "한국어"]
        },
        {
            "id": "paper2",
            "title": "과학기술 문헌 기반 지식 그래프 구축 및 활용 방안",
            "abstract": "본 논문에서는 과학기술 문헌을 활용한 지식 그래프 구축 방법론을 제안한다. 구축된 지식 그래프는 연구 동향 분석, 연구자 네트워크 분석 등 다양한 분야에 활용될 수 있다.",
            "authors": ["박지훈", "최민지"],
            "year": 2023,
            "keywords": ["지식 그래프", "과학기술 문헌", "정보 추출", "온톨로지"]
        },
        {
            "id": "paper3",
            "title": "미세먼지 측정 센서 네트워크 구축 및 오염지도 개발",
            "abstract": "본 연구에서는 도시 지역의 미세먼지 농도를 효과적으로 모니터링하기 위한 센서 네트워크 구축 방안과 실시간 오염지도 개발 방법을 제안한다. 제안된 시스템은 저비용 센서를 활용하여 고해상도 공간 데이터를 생성한다.",
            "authors": ["임성호", "박성준", "김효진"],
            "year": 2023,
            "keywords": ["미세먼지", "센서네트워크", "오염지도", "사물인터넷", "환경모니터링"]
        },
        {
            "id": "paper4",
            "title": "심층 신경망 기반 환자 건강 예측 모델",
            "abstract": "본 연구는 심층 신경망을 활용하여 환자의 건강 상태를 예측하는 모델을 제안한다. 전자의무기록(EMR) 데이터를 활용하여 다양한 질병의 발생 위험을 예측하는 방법을 제시한다.",
            "authors": ["정민수", "강다현"],
            "year": 2021,
            "keywords": ["심층 신경망", "헬스케어", "예측 모델", "전자의무기록"]
        },
        {
            "id": "paper5",
            "title": "한국어 과학기술 문헌의 자동 분류 시스템 개발",
            "abstract": "본 연구에서는 KorSci BERT를 활용한 한국어 과학기술 문헌 자동 분류 시스템을 개발하였다. 과학기술 분야의 특화된 어휘와 문맥을 반영하여 높은 분류 정확도를 달성하였으며, 분야별 논문 추천 시스템에 활용 가능하다.",
            "authors": ["이지원", "박태민"],
            "year": 2022,
            "keywords": ["문서 분류", "과학기술 문헌", "BERT", "기계학습", "자연어처리"]
        }
    ]

# 샘플 데이터 생성 및 저장
sample_papers = create_sample_papers()
sample_papers_path = f"{HBN_DATASET_DIR_PATH}/sample_papers.json"

with open(sample_papers_path, 'w', encoding='utf-8') as f:
    json.dump(sample_papers, f, ensure_ascii=False, indent=2)

print(f"샘플 논문 데이터가 {sample_papers_path}에 저장되었습니다.")

샘플 논문 데이터가 /content/drive/MyDrive/Workspaces/Hibrainnet/datasets/samples/sample_papers.json에 저장되었습니다.


In [ ]:
def run_search_demo(papers_path=f"{HBN_DATASET_DIR_PATH}/papers.json"):
    """
    검색 데모 실행

    Args:
        papers_path: 논문 데이터 파일 경로
    """
    # KorSci BERT 모델 경로 설정
    model_dir = HBN_MODEL_DIR_PATH

    # 검색 엔진 초기화
    search_engine = KorSciBertSearch(model_dir)

    try:
        # 논문 데이터 로드 및 임베딩 생성
        search_engine.load_papers(papers_path)
        search_engine.create_paper_embeddings()

        # 검색 수행
        test_queries = [
            "자연어처리 기술을 활용한 과학기술 문헌 분석",
            "미세먼지 측정 및 모니터링 방법",
            "딥러닝 기반 의료 데이터 분석"
        ]

        for i, query in enumerate(test_queries, 1):
            print(f"\n\n===== 검색 쿼리 {i}: {query} =====")
            results = search_engine.search(query, top_k=5)

            # 결과를 유사도 내림차순으로 정렬 (이미 정렬되어 있지만 명시적으로 재정렬)
            results = sorted(results, key=lambda x: x['similarity'], reverse=True)

            # 결과 출력
            for j, result in enumerate(results, 1):
                paper = result['paper']
                print(f"\n[{j}] {paper['title']} (유사도: {result['similarity']:.4f})")
                print(f"저자: {', '.join(paper['authors'])}")
                print(f"연도: {paper['year']}")
                print(f"키워드: {', '.join(paper['keywords'])}")
                abstract = paper['abstract']
                print(f"초록: {abstract[:150]}..." if len(abstract) > 150 else f"초록: {abstract}")

    finally:
        search_engine.close()

# 데모 실행 (저장된 샘플 데이터 파일 사용)
run_search_demo()

KISTI 토크나이저 초기화 완료 (Mecab)
총 5개의 논문 데이터를 로드했습니다.
논문 임베딩 생성 중...
임베딩 생성 완료. 형태: (5, 86)


===== 검색 쿼리 1: 자연어처리 기술을 활용한 과학기술 문헌 분석 =====

[1] 한국어 과학기술 문헌의 자동 분류 시스템 개발 (유사도: 0.5207)
저자: 이지원, 박태민
연도: 2022
키워드: 문서 분류, 과학기술 문헌, BERT, 기계학습, 자연어처리
초록: 본 연구에서는 KorSci BERT를 활용한 한국어 과학기술 문헌 자동 분류 시스템을 개발하였다. 과학기술 분야의 특화된 어휘와 문맥을 반영하여 높은 분류 정확도를 달성하였으며, 분야별 논문 추천 시스템에 활용 가능하다.

[2] 과학기술 문헌 기반 지식 그래프 구축 및 활용 방안 (유사도: 0.5009)
저자: 박지훈, 최민지
연도: 2023
키워드: 지식 그래프, 과학기술 문헌, 정보 추출, 온톨로지
초록: 본 논문에서는 과학기술 문헌을 활용한 지식 그래프 구축 방법론을 제안한다. 구축된 지식 그래프는 연구 동향 분석, 연구자 네트워크 분석 등 다양한 분야에 활용될 수 있다.

[3] 한국어 자연어처리를 위한 BERT 기반 임베딩 모델 연구 (유사도: 0.2468)
저자: 김철수, 이영희
연도: 2022
키워드: 자연어처리, BERT, 임베딩, 한국어
초록: 본 연구에서는 한국어 자연어처리를 위한 BERT 기반 임베딩 모델을 제안한다. 제안된 모델은 대용량 한국어 코퍼스를 사용하여 사전학습되었으며, 다양한 자연어처리 태스크에서 우수한 성능을 보인다.

[4] 심층 신경망 기반 환자 건강 예측 모델 (유사도: 0.0304)
저자: 정민수, 강다현
연도: 2021
키워드: 심층 신경망, 헬스케어, 예측 모델, 전자의무기록
초록: 본 연구는 심층 신경망을 활용하여 환자의 건강 상태를 예측하는 모델을 제안한다. 전자의무기록(EMR) 데이터를 활용하여 다양한 질병의 발생 위험을 예측하는 방법을 제시한다.

[5] 미세먼지 측정 센서 네